In [2]:
import sys
import pathlib
import pandas as pd

ROOT = pathlib.Path().absolute().parent.as_posix()
if ROOT not in sys.path:
    sys.path.append(ROOT)
    
from helpers import *

In [2]:
points_df = get_spaced_point_set_in_bbox(0.25, (51.239405, -0.625211), (51.737184, 0.328289))

221


100%|██████████| 221/221 [00:00<00:00, 736.04it/s]


In [3]:
points_df['Magnitude'] = pd.Series([1 for i in range(len(points_df))])

In [4]:
points_df.to_csv(ROOT + '/Spikes/Dash/data/points_df.csv')

In [5]:
points_df

,Latitude,Longitude,Magnitude
0,51.737184,-0.620643,1
1,51.737184,-0.617012,1
2,51.737184,-0.613382,1
3,51.737183,-0.609751,1
4,51.737183,-0.606120,1
...,...,...,...
58243,51.238843,0.312049,1
58244,51.238815,0.315640,1
58245,51.238786,0.319231,1
58246,51.238757,0.322822,1


In [4]:
points_df_aq_filled = pd.read_csv(ROOT + '/Spikes/Dash/data/points_df.csv', index_col = 0)

#constants
co_molar_mass = 28.01
no2_molar_mass = 46.0055
o3_molar_mass = 48
so2_molar_mass = 64.066

#apply aq functions to each row (using latitude and longitude columns) and multiply by associated molar mass to give g/m2
points_df_aq_filled['Value_co'] = points_df_aq_filled.apply(lambda row : co_function(row[0], row[1]) * co_molar_mass, axis=1)
points_df_aq_filled['Value_no2'] = points_df_aq_filled.apply(lambda row : no2_function(row[0], row[1]) * no2_molar_mass, axis=1)
points_df_aq_filled['Value_o3'] = points_df_aq_filled.apply(lambda row : o3_function(row[0], row[1]) * o3_molar_mass, axis=1)
points_df_aq_filled['Value_so2'] = points_df_aq_filled.apply(lambda row : so2_function(row[0], row[1]) * so2_molar_mass, axis=1)

#save dataframe
points_df_aq_filled.to_csv(ROOT + '/Spikes/Dash/data/points_df_aq_filled.csv')

#normalise each gas value set between 1 and 0 where 0 = 0% and 1 = 25%
for i in points_df_aq_filled.columns[3:7]:   #normalise columns 3-6, i.e. aq values
    points_df_aq_filled[i]=(points_df_aq_filled[i]-points_df_aq_filled[i].min())/(points_df_aq_filled[i].max()-points_df_aq_filled[i].min())

#calculate an aqi metric
#assumption: each gas is worth 25% of AQI, 100 / 4 gases
def calculate_aqi(aq1, aq2, aq3, aq4):
    #smaller value = better air quality
    aqi = (aq1 * (25/100)) + (aq2 * (25/100)) + (aq3 * (25/100)) + (aq4 * (25/100))
    return aqi

#apply calculate_aqi function to each row of the 4 aq columns
points_df_aq_filled['AQI'] = points_df_aq_filled.apply(lambda row : calculate_aqi(row[3], row[4], row[5], row[6]), axis=1)

#save dataframe
points_df_aq_filled.to_csv(ROOT + '/Spikes/Dash/data/points_df_aqindex_filled.csv')

In [5]:
points_df_aq_filled

,Latitude,Longitude,Magnitude,Value_co,Value_no2,Value_o3,Value_so2,AQI
0,51.737184,-0.620643,1,0.495930,0.296883,0.418262,0.334189,0.386316
1,51.737184,-0.617012,1,0.496156,0.302131,0.419181,0.334270,0.387934
2,51.737184,-0.613382,1,0.496412,0.302440,0.419523,0.334763,0.388284
3,51.737183,-0.609751,1,0.497956,0.302258,0.419939,0.333500,0.388413
4,51.737183,-0.606120,1,0.496061,0.302292,0.420329,0.331846,0.387632
...,...,...,...,...,...,...,...,...
58243,51.238843,0.312049,1,0.495346,0.158682,0.457220,0.293552,0.351200
58244,51.238815,0.315640,1,0.497392,0.159788,0.457003,0.294751,0.352233
58245,51.238786,0.319231,1,0.499898,0.160154,0.456728,0.295787,0.353142
58246,51.238757,0.322822,1,0.500813,0.160543,0.457355,0.295649,0.353590


#### NORMALISATION REQUIRES ENTIRE DATASET, NOT JUST ONE POINT AS BELOW CODE EXEMPLIFIES

In [ ]:
#constants
    co_molar_mass = 28.01
    no2_molar_mass = 46.0055
    o3_molar_mass = 48
    so2_molar_mass = 64.066
    
    #call aq functions and multiply by molar mass to convert mol/m2 into g/m2
    co = co_function(lat, lon) * co_molar_mass
    no2 = no2_function(lat, lon) * no2_molar_mass
    o3 = o3_function(lat, lon) * o3_molar_mass
    so2 = so2_function(lat, lon) * so2_molar_mass
    
    df = pd.DataFrame({'Value_co': [co], 'Value_no2': [no2], 'Value_o3': [o3], 'Value_so2': [so2]})
    print(df)
    
    for i in df.columns:
        df[i]=(df[i]-df[i].min())/(df[i].max()-df[i].min())
    print(df)
        
    def calculate_aqi(aq1, aq2, aq3, aq4):
        #smaller value = better air quality
        aqi = (aq1 * (25/100)) + (aq2 * (25/100)) + (aq3 * (25/100)) + (aq4 * (25/100))
        return aqi
    
    #apply calculate_aqi function to each row of the 4 aq columns
    df['AQI'] = df.apply(lambda row : calculate_aqi(row[0], row[1], row[2], row[3]), axis=1)
    print(df)
    return df['AQI'].item()